In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch.quantization
from torch.utils.data import DataLoader, TensorDataset
import time
import sys

In [46]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.MNIST(root="~/torch_datasets", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="~/torch_datasets", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [47]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input image
        out = self.linear(x)
        return out

# Set input/output dimensions for MNIST
input_dim = 28 * 28  # 28x28 pixels
output_dim = 10  # 10 classes

# Initialize the logistic regression model
model = LogisticRegressionModel(input_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 0.01)


In [48]:
# Training loop
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/5], Loss: 0.3717
Epoch [2/5], Loss: 0.4222
Epoch [3/5], Loss: 0.1103
Epoch [4/5], Loss: 0.3555
Epoch [5/5], Loss: 0.1049


In [49]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels.numpy())

# Report accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Original Model Accuracy: {accuracy:.4f}")

Original Model Accuracy: 0.9109


In [50]:
# Get model size in bytes
def get_model_size(model):
    total_size = 0
    for param in model.parameters():
        total_size += param.nelement() * param.element_size()
    return total_size

model_size = get_model_size(model)
print(f"Original Model Size: {model_size} bytes")

Original Model Size: 31400 bytes


In [51]:
# Measure inference time
start_time = time.time()
with torch.no_grad():
    for images, _ in test_loader:
        outputs = model(images)
inference_time = time.time() - start_time
print(f"Inference Time: {inference_time:.6f} seconds")

Inference Time: 1.369716 seconds


In [52]:
def model_size(model, quantized=False):
  total_size = 0
  total_params = 0
  if quantized==False:
     for layer in model.children():
        if hasattr(layer, 'weight') and layer.weight is not None:
          weight_tensor = layer.weight.data
          weight_size = len(weight_tensor.flatten())
          total_params+= weight_size*4


        if hasattr(layer, 'bias') and layer.bias is not None:
          bias_tensor = layer.bias.data
          bias_size = len(bias_tensor.flatten())
          total_params+= bias_size*4

  else:
    for layer in model.children():
      if hasattr(layer, 'weight') and layer.weight is not None:
        weight_tensor = layer.weight()
        weight_size = len(weight_tensor.flatten())
        total_params+= weight_size*1


      if hasattr(layer, 'bias') and layer.bias is not None:
        bias_tensor = layer.bias()
        bias_size = len(bias_tensor.flatten())
        total_params+= bias_size*4

  return total_params, total_size

In [53]:
# For the original model
params, size = model_size(model, quantized=False)
print(f"Original Model size: {params}")
quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
# For the quantized model
quant_params, quant_size = model_size(quantized_model, quantized=True)
print(f"Quantized Model size: {quant_params}")

Original Model size: 31400
Quantized Model size: 7880


In [54]:
# Quantization (Dynamic Quantization in PyTorch)
quantized_model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

# Measure inference time for quantized model
start_time = time.time()
with torch.no_grad():
    for images, _ in test_loader:
        outputs = quantized_model(images)
quantized_inference_time = time.time() - start_time
print(f"Quantized Inference Time: {quantized_inference_time:.6f} seconds")

Quantized Inference Time: 1.669748 seconds


In [55]:
# Evaluate quantized model
quantized_preds = []
with torch.no_grad():
    for images, _ in test_loader:
        outputs = quantized_model(images)
        _, predicted = torch.max(outputs, 1)
        quantized_preds.extend(predicted.numpy())

quantized_accuracy = accuracy_score(all_labels, quantized_preds)
print(f"Quantized Model Accuracy: {quantized_accuracy:.4f}")

Quantized Model Accuracy: 0.9102


In [56]:
# Model size comparison
print(f"Model Size Comparison:\nOriginal Model: {params} bytes\nQuantized Model: {quant_params} bytes")
print(f"\nAccuracy Comparison:\nOriginal Model: {accuracy*100:.2f}%\nQuantized Model: {quantized_accuracy*100:.2f}%")
print(f"\nInference Time Comparison:\nOriginal Model: {inference_time:.6f} seconds\nQuantized Model: {quantized_inference_time:.6f} seconds")


Model Size Comparison:
Original Model: 31400 bytes
Quantized Model: 7880 bytes

Accuracy Comparison:
Original Model: 91.09%
Quantized Model: 91.02%

Inference Time Comparison:
Original Model: 1.369716 seconds
Quantized Model: 1.669748 seconds
